# Обучение модели

### Установка и импорт неообходимых модулей

In [1]:
!pip install pandas sklearn catboost tqdm xgboost

In [2]:
import numpy as np
import pickle
from tqdm import tqdm
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Загрузка данных

In [3]:
df = pd.read_csv("uncoded_data.csv", index_col=0)
target = 'price'

## Препроцессинг

###  Кодирование нечисловых значений

In [4]:
les = {}
for name in tqdm(df.columns):
    if df[name].dtypes == object or df[name].dtypes == bool:
        df[name].fillna("-9999999")
        les[name] = preprocessing.LabelEncoder()
        df[name] = les[name].fit_transform(df[name].apply(str))

100%|██████████| 16/16 [00:00<00:00, 6254.90it/s]


In [5]:
df = df.dropna()

#### Сохранение моделей кодировшиков для разкодирования в будуюшем

In [6]:
with open('les.pkl', 'wb') as output:
    pickle.dump(les, output)

# Обучение модели

Определим функцию для разделения датасета на тестовую и тренировчную выборку для независимого тестирования точности модели

In [7]:
from sklearn.model_selection import train_test_split

def split_data(df, target):
    target_data = df[target]
    features_data = df.drop(target,axis=1)
    return train_test_split(features_data, target_data, test_size=0.15)

In [8]:
# Подключение библиотек с моделями
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [9]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

Подберем параметры которые максиально улучшат и оптимизируют процесс обучения,  
создадим функцию для удобного тестирования моделей

In [10]:

def test_models(x_train, x_test, y_train, y_test):
    models_list = [
        XGBRegressor(),
        CatBoostRegressor(silent=True),
        RandomForestRegressor(),
        KNeighborsRegressor(),
    ]

    for model in tqdm(models_list):
        score = cross_val_score(model, x_train, y_train)
        #model.fit(x_train, y_train)
        tqdm.write("\nТочность кросс-валидации" + str(type(model).__name__) + ": " + str(model.score(x_test, y_test)))
        tqdm.write("\nТочность " + str(type(model).__name__) + ": " + str(model.score(x_test, y_test)))
    return models_list*split_data(df, target))*split_data(df, target))

Обучим три модели и сравним их показатели

In [11]:
models_list = test_models(*split_data(df, target))

25%|██▌       | 1/4 [00:18<00:56, 18.76s/it]
Точность XGBRegressor: -3318.2816985510444%
 50%|█████     | 2/4 [00:51<00:45, 22.92s/it]
Точность CatBoostRegressor: -80.77932994110529%
 75%|███████▌  | 3/4 [04:50<01:27, 87.77s/it]
Точность RandomForestRegressor: -75.86963998489176%
100%|██████████| 4/4 [04:56<00:00, 74.05s/it]
Точность KNeighborsRegressor: -157.30406155792508%



In [12]:
for model in models_list:
    print("Точность " + str(type(regressor).__name__) + ": " + str(regressor.score(x_test, y_test)))

NameError: name 'regressor' is not defined

Посмотрев на результаты тестирования моделей можно сказать, что метод градиентного бустинга и метода сучайных деревьев очень схожи по точности, а точность метода ближайших соседей заметно меньше остальных двух.  Схожесть методов градиентного бустинга и случайных деревьев можно обьяснить, концептуальной схожестью алгоритмов: оба строят логические деревья.

# Feature engenering

## Обучение моделей на обработаных данных

In [ ]:
models_list = test_models(*split_data(df, target))

Как видно из вывода выше, реинженеринг фич помог увеличить точность моей целевой модели (градиентный бустинг) на 3%. На таком небольшом наборе данных это очень солидный прирост точности, даже не смотря на то, что точность метода ближайших соседей уменьшилась. Все вышеперечисленное доказывает, что выбраная мной модель действительно самая подходящая для данных задач

## Сохранение лучшей модели для дальнейшего использования

In [42]:
with open('model.pkl', 'wb') as output:
    pickle.dump(models_list[0], output)